In [1]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [4]:
arg_date = '2022-01-04'
src_format = '%Y-%m-%d'
src_bucket = 'demo-s3-nhhung2'
trg_bucket = 'xetra-112233'
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [3]:
arg_date= '2022-01-03'

In [5]:
arg_date_dt= datetime.strptime(arg_date, src_format).date() - timedelta(days=1)

In [6]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(src_bucket)
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[1], src_format).date() >= arg_date_dt]

In [7]:
objects

[s3.ObjectSummary(bucket_name='demo-s3-nhhung2', key='data/2022-01-03/2022-01-03_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='demo-s3-nhhung2', key='data/2022-01-03/2022-01-03_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='demo-s3-nhhung2', key='data/2022-01-03/2022-01-03_BINS_XETR10.csv'),
 s3.ObjectSummary(bucket_name='demo-s3-nhhung2', key='data/2022-01-03/2022-01-03_BINS_XETR11.csv'),
 s3.ObjectSummary(bucket_name='demo-s3-nhhung2', key='data/2022-01-03/2022-01-03_BINS_XETR12.csv'),
 s3.ObjectSummary(bucket_name='demo-s3-nhhung2', key='data/2022-01-03/2022-01-03_BINS_XETR13.csv'),
 s3.ObjectSummary(bucket_name='demo-s3-nhhung2', key='data/2022-01-03/2022-01-03_BINS_XETR14.csv'),
 s3.ObjectSummary(bucket_name='demo-s3-nhhung2', key='data/2022-01-03/2022-01-03_BINS_XETR15.csv'),
 s3.ObjectSummary(bucket_name='demo-s3-nhhung2', key='data/2022-01-03/2022-01-03_BINS_XETR16.csv'),
 s3.ObjectSummary(bucket_name='demo-s3-nhhung2', key='data/2022-01-04/2022-01-04_BINS_XETR08.csv'),


In [8]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    return df
df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index=True)

In [9]:
df_all = df_all.loc[:, columns]

In [10]:
df_all.dropna(inplace=True)

In [11]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

In [12]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

In [13]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

In [14]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [15]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [16]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [17]:
df_all = df_all.round(decimals=2)

In [18]:
df_all = df_all[df_all.Date >= arg_date]

In [19]:
from io import BytesIO

In [20]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket(trg_bucket)
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='xetra-112233', key='xetra_daily_report_20240122_202734.parquet')

In [24]:
for obj in bucket_target.objects.all():
    print(obj.key)

meta_file.csv
xetra_daily_report_20231216_105737.parquet
xetra_daily_report_20231216_171323.parquet
xetra_daily_report_20231218_160058.parquet
xetra_daily_report_20240122_202734.parquet


In [25]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20240122_202734.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)


In [26]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-01-04,37.75,37.85,37.75,37.85,27,1.20
1,AT000000STR1,2022-01-05,37.70,37.20,37.20,37.70,800,-1.72
2,AT000000STR1,2022-01-06,36.60,37.30,36.60,37.30,140,0.27
3,AT000000STR1,2022-01-07,36.85,37.35,36.85,37.35,475,0.13
4,AT000000STR1,2022-01-10,37.40,36.60,36.50,37.40,64,-2.01
...,...,...,...,...,...,...,...,...
51120,XS2376095068,2022-01-19,36.29,36.76,36.29,37.09,0,0.80
51121,XS2376095068,2022-01-20,36.89,38.13,36.89,38.13,54,3.74
51122,XS2376095068,2022-01-21,34.55,34.14,33.98,34.55,1857,-10.47
51123,XS2376095068,2022-01-24,30.94,30.26,29.39,31.07,5750,-11.35
